In [1]:
# import required libriaries
from pathlib import Path
import json

from fpl_predictor.util import *

In [2]:
# path to data directory
#path = Path('/home/jupyter/fpl-prediction/data')
path = Path('./fpl_predictor/data')

In [3]:
# read in latest training set
df_train = pd.read_csv(path/'train_v8.csv', index_col=0, 
                       dtype={'season':str,
                              'comp':str,
                              'squad':str})

# path to current season
season_paths = [path/'2021-22']

# name for current season
season_names = ['2122']

# team codes
teams = pd.read_csv(path/'teams.csv')

# latest remaining season dataset, for latest week's play_proba values
remaining_season = pd.read_csv(path/'remaining_season.csv', index_col=0)

# create players dataset
all_players = build_players(path, season_paths, season_names, teams)

# 2019-20 market value
# table was in slightly different position for 2019-20 season
# should check that 2019-20 season values remain the same once season starts
teams_mv = build_season_mv('2122', 11, range(13,33))

In [63]:
all_players#[all_players['full_name'] == 'Ben Davies']

,full_name,play_proba_2122,position_2122,id_2122,cost_2122,team_2122,web_name_2122
0,Bernd Leno,None,1,1,50,3,Leno
1,Rúnar Alex Rúnarsson,None,1,2,40,3,Rúnarsson
2,Willian Borges Da Silva,None,3,3,65,3,Willian
3,Pierre-Emerick Aubameyang,None,4,4,100,3,Aubameyang
4,Cédric Soares,None,2,5,45,3,Cédric
...,...,...,...,...,...,...,...
487,Yerson Mosquera Valdelamar,None,2,446,45,39,Mosquera
488,John Ruddy,None,1,452,45,39,Ruddy
489,Francisco Machado Mota de Castro Trincão,None,3,461,60,39,Trincão
490,Rayan Ait Nouri,None,2,470,45,39,Ait Nouri


In [50]:
df_train.tail(50)

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,crowds
114753,Conor Coady,38,2,90,Wolverhampton Wanderers,Manchester United,0.915835,1.729194,True,1,...,0.0,0.0,0.0,68.0,74.0,91.9,62.0,0.0,0.0,True
114754,Matt Phillips,38,3,90,West Bromwich Albion,Leeds,0.276793,0.515553,False,2,...,0.2,1.0,0.0,15.0,21.0,71.4,31.0,2.0,4.0,True
114755,Matthew Longstaff,38,3,0,Newcastle United,Fulham,0.546308,0.477990,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
114756,Oliver McBurnie,38,4,0,Sheffield United,Burnley,0.312126,0.307548,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
114757,N'Golo Kanté,38,3,0,Chelsea,Aston Villa,2.015377,0.913957,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
114758,Sander Berge,38,3,0,Sheffield United,Burnley,0.312126,0.307548,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
114759,Paulo Gazzaniga,38,1,0,Tottenham Hotspur,Leicester City,1.568259,1.165630,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
114760,Jaden Philogene-Bidace,38,3,0,Aston Villa,Chelsea,0.913957,2.015377,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
114761,Danny Ings,38,4,83,Southampton,West Ham United,0.648198,0.619204,False,2,...,0.0,2.0,0.0,12.0,16.0,75.0,16.0,0.0,0.0,True
114762,Rob Holding,38,2,90,Arsenal,Brighton and Hove Albion,1.269633,0.645850,True,7,...,0.0,1.0,0.0,61.0,66.0,92.4,48.0,0.0,0.0,True


In [4]:
# find the latest gameweek
last_gw = df_train['gw'][df_train['season'] == season_names[0]].max()

if np.isnan(last_gw): 
    last_gw = 1 
else: 
    last_gw = last_gw + 1
    
last_gw

1

In [ ]:
# build training set for latest gameweek
df_latest = build_season(season_paths[0], season_names[0], all_players, teams, teams_mv, [last_gw])

In [23]:
df_latest.drop_duplicates(inplace=True)

In [24]:
df_latest.head(50)

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,selected,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season
0,Max Thompson,38,4,0,Burnley,Sheffield United,0.307548,0.312126,False,0,...,9819,0,1,0.0,-9,263,272,0,2021-05-23T15:00:00Z,2021
1,Willy Boly,38,2,90,Wolverhampton Wanderers,Manchester United,0.915835,1.729194,True,0,...,93387,2,1,1.0,-82,375,457,1,2021-05-23T15:00:00Z,2021
2,Ciaran Clark,38,2,0,Newcastle United,Fulham,0.546308,0.477990,False,0,...,13830,2,0,0.0,-695,617,1312,0,2021-05-23T15:00:00Z,2021
3,Luke Cundle,38,3,0,Wolverhampton Wanderers,Manchester United,0.915835,1.729194,True,0,...,1394,2,1,0.0,58,108,50,0,2021-05-23T15:00:00Z,2021
4,Matt Ritchie,38,3,90,Newcastle United,Fulham,0.546308,0.477990,False,9,...,8393,2,0,2.0,-45,873,918,0,2021-05-23T15:00:00Z,2021
5,Adrián San Miguel del Castillo,38,1,0,Liverpool,Crystal Palace,2.142270,0.424933,True,0,...,28179,0,2,0.0,-134,250,384,0,2021-05-23T15:00:00Z,2021
6,Rhys Williams,38,2,90,Liverpool,Crystal Palace,2.142270,0.424933,True,6,...,27995,0,2,30.0,7372,8596,1224,0,2021-05-23T15:00:00Z,2021
7,André-Frank Zambo Anguissa,38,3,45,Fulham,Newcastle United,0.477990,0.546308,True,1,...,102838,2,0,4.0,-544,1824,2368,0,2021-05-23T15:00:00Z,2021
8,Jean Michaël Seri,38,3,0,Fulham,Newcastle United,0.477990,0.546308,True,0,...,2171,2,0,0.0,-5,0,5,0,2021-05-23T15:00:00Z,2021
9,Dara O'Shea,38,2,59,West Bromwich Albion,Leeds,0.276793,0.515553,False,0,...,13904,1,3,0.0,-250,110,360,0,2021-05-23T15:00:00Z,2021


In [25]:
df_latest.shape

(713, 34)

In [26]:
last_play_proba = remaining_season[remaining_season['gw'] == last_gw][['player', 'team', 'play_proba']]

In [27]:
last_play_proba.head()

,player,team,play_proba
0,Mesut Özil,Arsenal,0.0
1,Sokratis Papastathopoulos,Arsenal,0.0
2,David Luiz Moreira Marinho,Arsenal,0.5
3,Pierre-Emerick Aubameyang,Arsenal,1.0
4,Cédric Soares,Arsenal,1.0


In [28]:
# add latest week's play_proba for each player
df_latest = df_latest.merge(last_play_proba, on=['player', 'team'], how='left')

In [29]:
df_latest.drop_duplicates(inplace=True)

In [30]:
df_train.drop_duplicates(inplace=True)

In [31]:
if last_gw==1:
    df_latest['relative_market_value_team_season'] = df_latest['relative_market_value_team']
    df_latest['relative_market_value_opponent_team_season'] = df_latest['relative_market_value_opponent_team']

In [ ]:
# for when you're doing the season value properly
# relative_market_value_season = train_df[train_df['season'] == '2021'][['team', 'relative_market_value_team']].groupby(['team']).first()
# relative_market_value_season = relative_market_value_season.rename(columns={'relative_market_value_team': 'season_value'})
# relative_market_value_season

In [32]:
# now get futher stats from fbref
# standard stats
features = ['squad', 'comp', 'round', 'shots_total', 'shots_on_target', 'touches', 'pressures',
            'tackles', 'interceptions', 'blocks', 'xg', 'npxg', 'xa', 'sca', 
            'gca', 'passes_completed', 'passes', 'passes_pct',  'carries', 
            'dribbles_completed', 'dribbles']

In [35]:
# url for the premier league
url_base = 'https://fbref.com'
pl_history_page = '/en/comps/9/history/Premier-League-Seasons'
url = url_base + pl_history_page

# scrape the history table to get latest season
pl_history_table = get_table(url)
pl_history_urls = get_urls(['season'], 'season', pl_history_table, url_base)
season_urls = pl_history_urls[0:1]

df_enhanced = get_stats(features, season_urls, url_base)

2020-2021
https://fbref.com/en/comps/9/Premier-League-Stats
Manchester City
https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats
Ederson
Rúben Dias
Rodri
Raheem Sterling
João Cancelo
Bernardo Silva
İlkay Gündoğan
Kevin De Bruyne
Riyad Mahrez
Gabriel Jesus
Kyle Walker
John Stones
Phil Foden
Oleksandr Zinchenko
Ferrán Torres
Aymeric Laporte
Fernandinho
Benjamin Mendy
Nathan Aké
Sergio Agüero
Eric García
Scott Carson
Zack Steffen
Liam Delap
Manchester Utd
https://fbref.com/en/squads/19538871/Manchester-United-Stats
Bruno Fernandes
Aaron Wan-Bissaka
Harry Maguire
Marcus Rashford
Luke Shaw
Victor Lindelöf
Fred
David de Gea
Scott McTominay
Paul Pogba
Mason Greenwood
Anthony Martial
Edinson Cavani
Dean Henderson
Nemanja Matić
Daniel James
Eric Bailly
Alex Telles
Juan Mata
Donny van de Beek
Axel Tuanzebe
Brandon Williams
Amad Diallo
Anthony Elanga
Timothy Fosu-Mensah
Shola Shoretire
Odion Ighalo
Hannibal Mejbri
William Thomas Fish
Liverpool
https://fbref.com/en/squads/822bd0ba/Liverpool-

Dara O'Shea
Matt Phillips
Callum Robinson
Romaine Sawyers
Okay Yokuşlu
Jake Livermore
Grady Diangana
Ainsley Maitland-Niles
Mbaye Diagne
Karlan Grant
Kieran Gibbs
Branislav Ivanović
Robert Snodgrass
Filip Krovinović
Lee Peltier
Hal Robson-Kanu
Kamil Grosicki
Kyle Edwards
David Button
Ahmed Hegazi
Charlie Austin
Sam Field
Rekeem Harper
Sheffield Utd
https://fbref.com/en/squads/1df6b87e/Sheffield-United-Stats
Aaron Ramsdale
George Baldock
Chris Basham
Enda Stevens
John Egan
John Fleck
David McGoldrick
Oliver Norwood
Ethan Ampadu
John Lundstram
Ben Osborn
Oliver Burke
Sander Berge
Oliver McBurnie
Rhian Brewster
Jayden Bogle
Kean Bryan
Jack Robinson
Billy Sharp
Max Lowe
Phil Jagielka
Daniel Jebbison
Lys Mousset
Jack O'Connell
Iliman Ndiaye
Antwoine Hackford
Femi Seriki


In [36]:
# get the player name matching dictionary
with open(path/'match_list.json') as f:
  match_list = json.load(f)

match_list

{'Fabinho': 'Fabio Henrique Tavares',
 'Alisson': 'Alisson Ramses Becker',
 'Joël Matip': 'Joel Matip',
 'Joe Gomez': 'Joseph Gomez',
 'Naby Keïta': 'Naby Keita',
 'Adrián': 'Adrián San Miguel del Castillo',
 'Thiago Alcántara': 'Thiago Alcántara do Nascimento',
 'Allan': 'Allan Marques Loureiro',
 'Richarlison': 'Richarlison de Andrade',
 'Gylfi Sigurðsson': 'Gylfi Sigurdsson',
 'Séamus Coleman': 'Seamus Coleman',
 'André Gomes': 'André Filipe Tavares Gomes',
 'Bernard': 'Bernard Anício Caldeira Duarte',
 'Çağlar Söyüncü': 'Çaglar Söyüncü',
 'Bruno Fernandes': 'Bruno Miguel Borges Fernandes',
 'Fred': 'Frederico Rodrigues de Paula Santos',
 'Nemanja Matić': 'Nemanja Matic',
 'Alex Telles': 'Alex Nicolao Telles',
 'Pierre Højbjerg': 'Pierre-Emile Højbjerg',
 'Son Heung-min': 'Heung-Min Son',
 'Lucas Moura': 'Lucas Rodrigues Moura da Silva',
 'Érik Lamela': 'Erik Lamela',
 'Dele Alli': 'Bamidele Alli',
 'Carlos Vinícius': 'Carlos Vinicius Alves Morais',
 'Ederson': 'Ederson Santana de M

In [37]:
team_list = {}
team_list['Brighton'] = 'Brighton and Hove Albion'
team_list['Leeds United'] = 'Leeds'
team_list['Manchester Utd'] = 'Manchester United'
team_list['Newcastle Utd'] = 'Newcastle United'
team_list['Sheffield Utd'] = 'Sheffield United'
team_list['Tottenham'] = 'Tottenham Hotspur'
team_list['West Brom'] = 'West Bromwich Albion'
team_list['West Ham'] = 'West Ham United'
team_list['Wolves'] = 'Wolverhampton Wanderers'

In [38]:
# replace non-matching names/teams and extract gw as integer
df_enhanced['alt_name'] = df_enhanced['player'].replace(match_list)
df_enhanced['squad'] = df_enhanced['squad'].replace(team_list)
df_enhanced['round'] = df_enhanced['round'].str[-2:].astype('int')

In [39]:
# remove duplicated (happens when players transferred within season)
df_enhanced = df_enhanced.drop_duplicates()

In [40]:
df_enhanced

,season,player,date,squad,comp,round,shots_total,shots_on_target,touches,pressures,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,alt_name
0,2020-2021,Ederson,2020-09-21,Manchester City,Premier League,2,0,0,36,0,...,0.0,0,0,30,36,83.3,27,0,0,Ederson Santana de Moraes
1,2020-2021,Ederson,2020-09-27,Manchester City,Premier League,3,0,0,19,0,...,0.0,0,0,15,15,100.0,5,0,0,Ederson Santana de Moraes
2,2020-2021,Ederson,2020-10-03,Manchester City,Premier League,4,0,0,36,0,...,0.0,0,0,24,30,80.0,17,0,0,Ederson Santana de Moraes
3,2020-2021,Ederson,2020-10-17,Manchester City,Premier League,5,0,0,50,0,...,0.0,0,0,29,46,63.0,29,0,0,Ederson Santana de Moraes
4,2020-2021,Ederson,2020-10-24,Manchester City,Premier League,6,0,0,26,0,...,0.0,0,0,24,26,92.3,13,0,0,Ederson Santana de Moraes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10559,2020-2021,Jack O'Connell,2020-09-14,Sheffield United,Premier League,1,0,0,62,7,...,0.0,0,0,41,54,75.9,40,0,1,Jack O'Connell
10560,2020-2021,Jack O'Connell,2020-09-21,Sheffield United,Premier League,2,0,0,34,3,...,0.0,0,0,19,23,82.6,10,0,0,Jack O'Connell
10561,2020-2021,Iliman Ndiaye,2021-03-14,Sheffield United,Premier League,28,0,0,6,8,...,0.0,0,0,3,3,100.0,4,0,0,Iliman Ndiaye
10562,2020-2021,Antwoine Hackford,2021-01-02,Sheffield United,Premier League,17,1,0,3,3,...,0.0,0,0,1,1,100.0,0,0,0,Antwoine Hackford


In [41]:
# create date field in same format as enhanced data
df_latest['date'] = df_latest['kickoff_time'].str[:10]

In [50]:
# check if any names aren't matching
enhanced_players = df_enhanced[df_enhanced['date'].isin(df_latest['date'].unique())]['alt_name'].unique().tolist()
standard_players = df_latest['player'].unique().tolist()

unmatched_players = [x for x in enhanced_players if x not in standard_players]
unmatched_players

[]

In [51]:
# check if any teams aren't matching
enhanced_teams = df_enhanced[df_enhanced['date'].isin(df_latest['date'].unique())]['squad'].unique().tolist()
standard_teams = df_latest['team'].unique().tolist()

unmatched_teams = [x for x in enhanced_teams if x not in standard_teams]
unmatched_teams

[]

In [43]:
# any additional corrections
# match_list['William Thomas Fish'] = 'William Fish'
# match_list['Okay Yokuşlu'] = 'Okay Yokuslu'

# match_list['Abdoulaye Doucour�'] = 'Abdoulaye Doucouré'
# match_list['James Rodr�guez'] = 'James Rodríguez'
# match_list['Gylfi Sigur�sson'] = 'Gylfi Sigurdsson'
# match_list['Andr� Gomes'] = 'André Filipe Tavares Gomes'
# match_list['S�amus Coleman'] = 'Seamus Coleman'
# match_list['Emiliano Mart�nez'] = 'Emiliano Martínez'
# match_list['Bertrand Traor�'] = 'Bertrand Traoré'
# match_list['Tr�z�guet'] = 'Mahmoud Ahmed Ibrahim Hassan'
# match_list['H�lder Costa'] = 'Hélder Wander Sousa de Azevedo e Costa'
# match_list['Pablo Hern�ndez'] = 'Pablo Hernández Domínguez'
# match_list['Rui Patr�cio'] = 'Rui Pedro dos Santos Patrício'
# match_list['N�lson Semedo'] = 'Nélson Cabral Semedo'
# match_list['R�ben Neves'] = 'Rúben Diogo da Silva Neves'
# match_list['Jo�o Moutinho'] = 'João Filipe Iria Santos Moutinho'
# match_list['Adama Traor�'] = 'Adama Traoré'
# match_list['Romain Sa�ss'] = 'Romain Saïss'
# match_list['Rayan A�t Nouri'] = 'Rayan Ait Nouri'
# match_list['Fernando Mar�al'] = 'Fernando Marçal'
# match_list['F�bio Silva'] = 'Fabio Silva'
# match_list['Willian Jos�'] = 'Willian José Da Silva'
# match_list['Miguel Almir�n'] = 'Miguel Almirón'

In [44]:
# replace additional non-matching names
# df_enhanced['alt_name'] = df_enhanced['player'].replace(match_list)

In [48]:
# make sure there are no non matches after update
# enhanced_players =  df_enhanced[df_enhanced['round'] == last_gw]['alt_name'].unique().tolist()
# standard_players = df_latest['player'].unique().tolist()

# unmatched_players = [x for x in enhanced_players if x not in standard_players]
# unmatched_players

In [49]:
# if match list was updated then update the file
# json_file = json.dumps(match_list)
# f = open(path/"match_list.json","w")
# f.write(json_file)
# f.close()

In [52]:
# check there are no duplicates in the join fields for the enhanced data
assert(len(df_enhanced[df_enhanced.duplicated(subset=['alt_name', 'date'], keep=False)]) == 0)

In [53]:
df_latest[df_latest.duplicated(subset=['date','player','team'], keep=False)]

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season,play_proba,date


In [54]:
# check there are no duplicates in the join fields for the standard data
assert(len(df_train[df_train.duplicated(subset=['date','player','team'])]) == 0)

In [55]:
# df_train[df_train.duplicated(subset=['date','player','team'])]

In [56]:
# join standard and enhanced dataframes
df_latest_further = df_latest.merge(df_enhanced.drop(['player', 'season', 'round'], axis=1), 
                                   left_on=['date', 'player', 'team'], 
                                   right_on=['date', 'alt_name', 'squad'], how='left')

In [57]:
# drop the alt name field and check the length of the standard
# dataframe has not increased
df_latest_further = df_latest_further.drop(['alt_name'], axis=1)
assert(len(df_latest_further) == len(df_latest_further))

In [58]:
assert(sum(df_latest_further[df_latest_further['minutes'] > 0]['xg'].isnull()) == 0)

In [59]:
df_train_new = pd.concat([df_train, df_latest_further], ignore_index=True, axis=0, sort=False)

In [60]:
df_train_new[df_train_new['player'] == 'Harry Kane'].tail()

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,npxg,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles
111350,Harry Kane,34,4,90,Tottenham Hotspur,Sheffield United,1.587579,0.356319,True,2,...,0.9,0.2,6,0,31,37,83.8,42,5,6
112127,Harry Kane,35,4,90,Tottenham Hotspur,Leeds,1.586506,0.499628,False,2,...,0.1,0.3,4,0,26,35,74.3,23,0,0
113010,Harry Kane,36,4,90,Tottenham Hotspur,Wolverhampton Wanderers,1.586460,0.960312,True,8,...,1.5,0.1,6,1,18,27,66.7,28,3,5
113631,Harry Kane,37,4,90,Tottenham Hotspur,Aston Villa,1.586543,0.884857,True,2,...,0.1,0.2,1,0,11,24,45.8,21,1,1
114303,Harry Kane,38,4,90,Tottenham Hotspur,Leicester City,1.568259,1.165630,False,10,...,0.6,0.1,3,2,8,16,50.0,28,6,7


In [61]:
df_train_new.tail(7)

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,npxg,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles
114796,Max Kilman,38,2,0,Wolverhampton Wanderers,Manchester United,0.915835,1.729194,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114797,Jay-Roy Grot,38,3,0,Leeds,West Bromwich Albion,0.515553,0.276793,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114798,Aaron Ramsdale,38,1,90,Sheffield United,Burnley,0.312126,0.307548,True,9,...,0.0,0.0,1,0,19,41,46.3,18,0,0
114799,Adrien Silva,38,3,0,Leicester City,Tottenham Hotspur,1.165630,1.568259,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114800,Sidnei Tavares,38,3,0,Leicester City,Tottenham Hotspur,1.165630,1.568259,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114801,Kalvin Phillips,38,3,90,Leeds,West Bromwich Albion,0.515553,0.276793,True,6,...,0.0,0.0,2,0,39,47,83.0,48,1,4
114802,Raphael Dias Belloli,38,3,90,Leeds,West Bromwich Albion,0.515553,0.276793,True,5,...,0.1,0.4,5,1,32,47,68.1,40,2,6


In [62]:
df_latest_further.shape

(713, 56)

In [63]:
df_train_new.shape

(114803, 58)

In [64]:
# save latest training set to csv
# overwrite existing one
df_train_new.to_csv(path/'train_v8.csv')

In [50]:
# df_train_new[(df_train_new['team'] == 'Manchester United') & 
#              (df_train_new['gw'] == 32) & 
#              (df_train_new['season'] == '1920')]

In [35]:
latest_fixtures

,code,event,finished,finished_provisional,id,kickoff_time,minutes,provisional_start_time,started,team_a,team_a_score,team_h,team_h_score,stats,team_h_difficulty,team_a_difficulty,pulse_id
0,2210271,1,False,False,1,2021-08-13T19:00:00Z,0,False,False,1,NaN,3,NaN,[],3,2,66342
1,2210276,1,False,False,6,2021-08-14T11:30:00Z,0,False,False,10,NaN,13,NaN,[],3,4,66347
2,2210272,1,False,False,2,2021-08-14T14:00:00Z,0,False,False,4,NaN,5,NaN,[],2,2,66343
3,2210273,1,False,False,3,2021-08-14T14:00:00Z,0,False,False,7,NaN,6,NaN,[],2,4,66344
4,2210274,1,False,False,4,2021-08-14T14:00:00Z,0,False,False,16,NaN,8,NaN,[],2,3,66345
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,2210646,38,False,False,376,2022-05-22T15:00:00Z,0,False,False,13,NaN,7,NaN,[],4,2,66717
376,2210647,38,False,False,377,2022-05-22T15:00:00Z,0,False,False,16,NaN,9,NaN,[],2,3,66718
377,2210648,38,False,False,378,2022-05-22T15:00:00Z,0,False,False,20,NaN,11,NaN,[],2,4,66719
378,2210649,38,False,False,379,2022-05-22T15:00:00Z,0,False,False,2,NaN,12,NaN,[],2,5,66720


In [21]:
# get fixtures
fixtures = pd.read_csv(path/'2021-22/fixtures.csv')
team_dict = teams[['team_2122', 'team']].dropna().set_index('team_2122').to_dict()['team']

fixtures['home_team'] = fixtures['team_h'].replace(team_dict)
fixtures['away_team'] = fixtures['team_a'].replace(team_dict)
fixtures = fixtures[['home_team', 'away_team', 'kickoff_time', 'event']]
fixtures = fixtures.rename(columns={'event':'gw'})


# use these two lines if updating play_proba later in week
# current_gw = last_gw
# df_train_new = pd.read_csv(path/'train_v7.csv', index_col=0, dtype={'season':str,
#                                                                     'comp':str,
#                                                                     'squad':str})

# set starting gameweek (where are we right now in the season)
# current_gw = last_gw + 1
current_gw = 1
fixtures = fixtures[fixtures['gw'] >= current_gw].sort_values(by=['gw'])
fixtures['match_no'] = np.arange(fixtures.shape[0])

In [22]:
fixtures.head(50)

,home_team,away_team,kickoff_time,gw,match_no
0,Brentford,Arsenal,2021-08-13T19:00:00Z,1,0
1,Manchester United,Leeds,2021-08-14T11:30:00Z,1,1
2,Burnley,Brighton and Hove Albion,2021-08-14T14:00:00Z,1,2
3,Chelsea,Crystal Palace,2021-08-14T14:00:00Z,1,3
4,Everton,Southampton,2021-08-14T14:00:00Z,1,4
5,Leicester City,Wolverhampton Wanderers,2021-08-14T14:00:00Z,1,5
6,Watford,Aston Villa,2021-08-14T14:00:00Z,1,6
7,Norwich,Liverpool,2021-08-14T16:30:00Z,1,7
8,Newcastle United,West Ham United,2021-08-15T13:00:00Z,1,8
9,Tottenham Hotspur,Manchester City,2021-08-15T15:30:00Z,1,9


In [23]:
# add universal team codes for home and away teams
fixtures = fixtures.merge(teams, left_on='home_team', right_on='team', how='left')
fixtures = fixtures.merge(teams, left_on='away_team', right_on='team', how='left')
fixtures = fixtures[['gw', 'match_no', 'home_team', 'away_team', 'kickoff_time', 'team_code_x', 'team_code_y']]
fixtures.rename(index=str,
                columns={'team_code_x':'home_team_code',
                         'team_code_y':'away_team_code'},
                inplace=True)

In [24]:
fixtures.head()

,gw,match_no,home_team,away_team,kickoff_time,home_team_code,away_team_code
0,1,0,Brentford,Arsenal,2021-08-13T19:00:00Z,94,3
1,1,1,Manchester United,Leeds,2021-08-14T11:30:00Z,1,2
2,1,2,Burnley,Brighton and Hove Albion,2021-08-14T14:00:00Z,90,36
3,1,3,Chelsea,Crystal Palace,2021-08-14T14:00:00Z,8,31
4,1,4,Everton,Southampton,2021-08-14T14:00:00Z,11,20


In [25]:
# all_players.replace(to_replace='None', value=1, inplace=True)

In [26]:
all_players[all_players['play_proba_2122'] == '25']

,full_name,play_proba_2122,position_2122,id_2122,cost_2122,team_2122,web_name_2122
14,Thomas Partey,25,3,15,50,3,Partey
22,Gabriel Magalhães,25,2,23,50,3,Gabriel
97,Tariq Lamptey,25,2,71,45,36,Lamptey
102,Danny Welbeck,25,4,447,60,36,Welbeck
109,Dale Stephens,25,3,102,45,90,Stephens
110,Kevin Long,25,2,103,40,90,Long
303,Marcus Rashford,25,3,281,95,1,Rashford
319,Martin Dubravka,25,1,295,45,4,Dubravka


In [27]:
# join home team to all players for current season
home_df = fixtures.merge(all_players, 
               left_on='home_team_code', 
               right_on='team_2122', 
               how='left')

# pull out the required fields and rename columns
home_df = home_df[['gw', 'match_no', 'home_team', 'away_team', 'kickoff_time', 
                   'full_name', 'position_2122', 'cost_2122', 'play_proba_2122', 
                   'web_name_2122']]
home_df.rename(index=str, 
               columns={'home_team':'team',
                        'away_team':'opponent_team',
                        'full_name':'player',
                        'position_2122':'position',
                        'cost_2122':'price',
                        'play_proba_2122':'play_proba',
                        'web_name_2122':'short_name'},
              inplace=True)

# add home flag
home_df['was_home'] = True

In [28]:
# join away team to all players for current season
away_df = fixtures.merge(all_players, 
               left_on='away_team_code', 
               right_on='team_2122', 
               how='left')

# pull out the required fields and rename columns
away_df = away_df[['gw',  'match_no', 'away_team', 'home_team', 'kickoff_time', 
                   'full_name', 'position_2122', 'cost_2122', 'play_proba_2122', 
                   'web_name_2122']]
away_df.rename(index=str, 
               columns={'away_team':'team',
                        'home_team':'opponent_team',
                        'full_name':'player',
                        'position_2122':'position',
                        'cost_2122':'price',
                        'play_proba_2122':'play_proba',
                        'web_name_2122':'short_name'},
              inplace=True)

# add home flag
away_df['was_home'] = False

In [29]:
# look at away players
away_df.head()

,gw,match_no,team,opponent_team,kickoff_time,player,position,price,play_proba,short_name,was_home
0,1,0,Arsenal,Brentford,2021-08-13T19:00:00Z,Bernd Leno,1,50,None,Leno,False
1,1,0,Arsenal,Brentford,2021-08-13T19:00:00Z,Rúnar Alex Rúnarsson,1,40,None,Rúnarsson,False
2,1,0,Arsenal,Brentford,2021-08-13T19:00:00Z,Willian Borges Da Silva,3,65,None,Willian,False
3,1,0,Arsenal,Brentford,2021-08-13T19:00:00Z,Pierre-Emerick Aubameyang,4,100,None,Aubameyang,False
4,1,0,Arsenal,Brentford,2021-08-13T19:00:00Z,Cédric Soares,2,45,None,Cédric,False


In [30]:
# look at home players
home_df.head()

,gw,match_no,team,opponent_team,kickoff_time,player,position,price,play_proba,short_name,was_home
0,1,0,Brentford,Arsenal,2021-08-13T19:00:00Z,Pontus Jansson,2,45,None,Jansson,True
1,1,0,Brentford,Arsenal,2021-08-13T19:00:00Z,Christian Nørgaard,3,50,None,Nørgaard,True
2,1,0,Brentford,Arsenal,2021-08-13T19:00:00Z,Ivan Toney,4,65,None,Toney,True
3,1,0,Brentford,Arsenal,2021-08-13T19:00:00Z,Tariqe Fosu-Henry,3,55,None,Fosu,True
4,1,0,Brentford,Arsenal,2021-08-13T19:00:00Z,David Raya Martin,1,45,None,Raya,True


In [31]:
# # calculate average minutes over last 5 gameweeks
# # first get last 5 gameweeks in current season
# recent_mins = df_train_new[(df_train_new['gw'] >= current_gw - 5) & 
#                            (df_train_new['season'] == season_names[0])][['player', 'minutes', 'play_proba']]

# # recent_mins = df_train_new[((df_train_new['gw'] >= 40 - 5) & 
# #                            (df_train_new['season'] == '1920')) | 
# #                            ((df_train_new['gw'] >= 1) & 
# #                            (df_train_new['season'] == '2021'))][['player', 'minutes', 'play_proba']]

# # exclude weeks where the player had an injury designation (i.e. keep where play_proba is 1)
# recent_mins = recent_mins[recent_mins['play_proba'] == 1]

# len(recent_mins)

# # average for each player
# recent_mins = recent_mins.groupby(['player'])['minutes'].mean().to_dict()

# recent_mins

In [32]:
# concatenate home and away players
remaining_season_df = home_df.append(away_df).reset_index(drop=True)

# add home and away team market values
remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '2122'],
                                                left_on='team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '2122'],
                                                left_on='opponent_team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df.rename(index=str, 
                           columns={'relative_market_value_x':'relative_market_value_team',
                                    'relative_market_value_y':'relative_market_value_opponent_team'},
                           inplace=True)

# add season name
remaining_season_df['season'] = season_names[0]

# divide cost by 10 for actual cost
remaining_season_df['price'] = remaining_season_df['price']/10

# set availability probability
# 0 = 0% chance, 25 = 25% chance, etc
# 'None' or '100' = 100% chance
remaining_season_df.loc[remaining_season_df['play_proba'] == 'None', 'play_proba'] = 100
remaining_season_df['play_proba'] = remaining_season_df['play_proba'].astype('float') / 100

# cast position to integer
remaining_season_df['position'] = remaining_season_df['position'].astype(int)

# # add minutes based on last five fixtures
# remaining_season_df['minutes'] = remaining_season_df['player'].map(recent_mins)
# remaining_season_df['minutes'].fillna(0, inplace=True)

# # multiply minutes by play probability
# remaining_season_df['minutes'] = remaining_season_df['minutes'] * remaining_season_df['play_proba']

In [33]:
remaining_season_df.head(50)

,gw,match_no,team,opponent_team,kickoff_time,player,position,price,play_proba,short_name,was_home,relative_market_value_team,relative_market_value_opponent_team,season
0,1,0,Brentford,Arsenal,2021-08-13T19:00:00Z,Pontus Jansson,2,4.5,1.00,Jansson,True,0.346219,1.246886,2122
1,1,0,Brentford,Arsenal,2021-08-13T19:00:00Z,Christian Nørgaard,3,5.0,1.00,Nørgaard,True,0.346219,1.246886,2122
2,1,0,Brentford,Arsenal,2021-08-13T19:00:00Z,Ivan Toney,4,6.5,1.00,Toney,True,0.346219,1.246886,2122
3,1,0,Brentford,Arsenal,2021-08-13T19:00:00Z,Tariqe Fosu-Henry,3,5.5,1.00,Fosu,True,0.346219,1.246886,2122
4,1,0,Brentford,Arsenal,2021-08-13T19:00:00Z,David Raya Martin,1,4.5,1.00,Raya,True,0.346219,1.246886,2122
5,1,0,Brentford,Arsenal,2021-08-13T19:00:00Z,Sergi Canós,3,5.5,1.00,Canós,True,0.346219,1.246886,2122
6,1,0,Brentford,Arsenal,2021-08-13T19:00:00Z,Pelenda Joshua Dasilva,3,5.5,1.00,Dasilva,True,0.346219,1.246886,2122
7,1,0,Brentford,Arsenal,2021-08-13T19:00:00Z,Rico Henry,2,4.5,1.00,Henry,True,0.346219,1.246886,2122
8,1,0,Brentford,Arsenal,2021-08-13T19:00:00Z,Vitaly Janelt,3,5.0,1.00,Janelt,True,0.346219,1.246886,2122
9,1,0,Brentford,Arsenal,2021-08-13T19:00:00Z,Saman Ghoddos,3,5.5,1.00,Ghoddos,True,0.346219,1.246886,2122


In [34]:
# save latest prediction set to csv
remaining_season_df.to_csv(path/'remaining_season.csv')